In [5]:
# 7/22/21: https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython

%load_ext autoreload
%autoreload 2

# end cite


In [6]:
import json
import os
from os.path import join, exists

import pandas as pd
import matplotlib.pyplot as plt

from utils import load_models, transformers_bert_completions, split_gen
from utils_child import child_models
import config

In [ ]:
# paths = [
#     split_gen.get_split_folder('all', 'all', config.model_dir),
#     split_gen.get_split_folder('age', 'young', config.model_dir),
#     split_gen.get_split_folder('age', 'old', config.model_dir)
# ]

paths = [
    split_gen.get_split_folder('child', 'Alex', config.model_dir),
]

has_tags_list = [
    True,
]

for path, tags in zip(paths, has_tags_list):

    tag_str =  "with_tags" if tags else "no_tags"

    with open(join(path, join(tag_str, 'trainer_state.json')), 'r') as f:

        gen_dict = json.load(f)
        this_history = gen_dict['log_history']

        epochs = []; losses = []

        for step_dict in this_history[:-1]: # Omit summary stats
            if 'eval_loss' not in step_dict: continue
            epochs.append(step_dict['epoch'])
            losses.append(step_dict['eval_loss'])

        train_epochs = []; train_losses = []
        for step_dict in this_history[:-1]: # Omit summary stats
            if 'loss' not in step_dict: continue
            train_epochs.append(step_dict['epoch'])
            train_losses.append(step_dict['loss'])

        print(gen_dict) # Because it's still on the debugging mode.

        this_step = gen_dict['best_model_checkpoint'].split('/')[-1]
        dataset_name = '/'.join(path.split('/')[-2:]) + f'/{tag_str} | step: {this_step}' + f"/{gen_dict['global_step']}"

    plt.title(f'{dataset_name}')
    plt.xlabel('epoch'); plt.ylabel('loss')
    plt.plot(epochs, losses, label = 'val')
    plt.plot(train_epochs, train_losses, label = 'train')

    plt.legend(); plt.show()


{'best_metric': 2.1568286418914795, 'best_model_checkpoint': '/net/vast-storage/scratch/vast/cpl/wongn/child_split/child-directed-listening/experiments/no_versioning/models/child/Alex/with_tags/checkpoint-20', 'epoch': 10.0, 'global_step': 20, 'is_hyper_param_search': False, 'is_local_process_zero': True, 'is_world_process_zero': True, 'log_history': [{'epoch': 5.0, 'learning_rate': 2.5e-05, 'loss': 2.425, 'step': 10}, {'epoch': 5.0, 'eval_loss': 2.3437743186950684, 'eval_runtime': 0.3523, 'eval_samples_per_second': 28.384, 'step': 10}, {'epoch': 10.0, 'learning_rate': 0.0, 'loss': 1.9643, 'step': 20}, {'epoch': 10.0, 'eval_loss': 2.1568286418914795, 'eval_runtime': 0.3551, 'eval_samples_per_second': 28.158, 'step': 20}, {'epoch': 10.0, 'step': 20, 'total_flos': 0, 'train_runtime': 19.2954, 'train_samples_per_second': 1.037}], 'max_steps': 20, 'num_train_epochs': 10, 'total_flos': 0, 'trial_name': None, 'trial_params': None}
